# <font color='black'>Методы анализа неоднородных данных и паттерн-анализ </font>
# <font color='black'>Практическое занятие 2 </font>


## <font color='black'>Модели с переменными взаимодействия </font>
На этом занятии для отработки моделей с переменными взаимодействия мы продолжим работать с данными из статьи [Kalenborn C., Lessman C., 2013](https://yadi.sk/i/nlEQUoWKiqY0UA). Одна из частей анализа в данной статье выполнена на основе cross-section data (использованы усредненные данные за 2005 - 2010 гг.). Авторы изучают взаимосвязь уровня коррупции и демократии, предполагая, что ее характер зависит от значений показателя свободы прессы. Кратко о данных:
* cpi - уровень коррупции: Corruption Perception Index. Непрерывная шкала от 0 до 10, где 10 означает наиболее высокий уровень коррупции.
* dem - индекс демократии: Vanhanen’s democratization index. Непрерывная шкала от 0 до 100, где 100 означает максимальное значение уровня демократии.
* fp - свобода прессы: Freedom House. Приведен к непрерывной шкале от 0 до 100, где 100 - наиболее высокое значение свободы прессы.
* loggdppc - натуральный логарифм ВВП на душу населения. World Bank.
* stab - уровень политической стабильности. Индекс построен на основе показателей "Political Stability" и "Absence of Violence/Terrorism" из the Worldwide Governance Indicators. Непрерывная шкала от -2.5 до 2.5, где 2.5 соответствует наиболее высокому уровню политической стабильности.
* britcol - дамми-переменная, где 1 - бывшая британская колония.

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import seaborn as sns
import numpy as np
import matplotlib.pyplot as mpl
%matplotlib inline

Откроем массив данных для репликации результатов исследования - lab1.dta.

In [ ]:
lab2 = pd.read_stata('lab1.dta')
lab2 = lab2.dropna()

Познакомимся с тем, как устроен массив данных.

In [ ]:
lab2.head(10)

Описательные статистики переменных особенно важны перед оцениванием моделей с переменными взаимодействия. Возможно, нам понадобится некоторые алгебраические преобразования для более удобной интерпретации предельных эффектов.

In [ ]:
lab2.describe()

Оценим первую наиболее простую регрессионную модель без переменных взаимодействия. Проинтерпретируйте все оценки коэффициентов модели m1.

In [ ]:
m1 = smf.ols(formula = "cpi ~ dem + fp + loggdppc + britcol + stab", data = lab2).fit()
print(m1.summary())

Рассмотрим, как различается уровень коррупции в группах стран с разным уровнем свободы прессы. Предварительно разделим страны на 2 группы: "not free" и "free". В качестве порогового значения для разделения возьмем 70 баллов по показателю "свобода прессы".

In [ ]:
def fp_new(series):
    if series <= 70: return 0
    elif series > 70: return 1

lab2['groups'] = lab2['fp'].apply(fp_new)


Модель 2 включает вместо непрерывной переменной уровня свободы прессы бинарный показатель "groups" в качестве предиктора. Проинтерпретируйте оценки коэффициентов модели m2.

In [ ]:
m2 = smf.ols(formula = "cpi ~ dem + groups + loggdppc + britcol + stab", data = lab2).fit()
print(m2.summary())

Тем не менее, данная модель не позволяет ответить на ключевой вопрос, которые ставили перед собой Kalenborn C., Lessman C. А именно, их интересовало, различается ли взаимосвязь коррупции и уровня демократии в странах с разными значениями свободы прессы? Для того, чтобы продвинуться в ответе на данный вопрос, представим следующий график. Какой предварительный вывод Вы можете сделать на основе данного графика?  

In [ ]:
sns.lmplot(x ='dem', y = 'cpi', hue = 'groups', data = lab2, ci = 95)

Изменим спецификацию регрессионной модели так, чтобы она позволяла ответить на вопрос, как различается взаимосвязь коррупции и уровня демократии в двух группах стран (условно назовем их "свободные" и "несвободные"). Для этого нужно ввести переменную взаимодействия между предиктором "dem" и "groups".
Проинтерпретируйте оценки коэффициентов в модели m3.

In [ ]:
m3 = smf.ols(formula = "cpi ~ dem*groups + loggdppc + britcol + stab", data = lab2).fit()
print(m3.summary())

Проиллюстрируем на графике различия между предельными эффектами демократии в зависимости от групп стран. Для этого нам предварительно понадобится
* задать формулу для расчета предельных эффектов. Для этого мы

1) извлечем оценки коэффициентов модели m3 (marg_effect)

2) получим вектор уникальных значений переменной "groups" (values), которая является условием (то есть, от ее значений зависит предельный эффект демократии)

* задать формулу для расчета стандартных ошибок

In [ ]:
print(m3.params)
print(m3.cov_params())

In [ ]:
values = lab2.groups.unique()
marg_effect = m3.params[1] + m3.params[3]*values
Cov_m3 = np.asmatrix(m3.cov_params())
se = (Cov_m3[1,1] + values**2*(Cov_m3[3,3]) + 2*values*Cov_m3[1,3])**(.5)
from scipy.stats import norm
fig, axes = mpl.subplots()
mpl.errorbar(values, marg_effect, yerr=norm.ppf(0.975)*se, fmt='bD', capsize = 10)
axes.set(xticks = [0,1])
axes.set_ylabel('Effect of democracy on corruption')
mpl.show()


Теперь попробуем оценить модель, но уже с переменной взаимодействия между исходными непрерывными переменными "dem" и "fp", без деления стран на группы "not free" и "free". Какой вывод можно сделать на основе оценок данной модели?
Предложите возможные преобразования для более удобной интерпретации и примените их при оценивании модели (в качестве дополнительной практики после занятия).

In [ ]:
m4 = smf.ols(formula = "cpi ~ dem*fp + loggdppc + britcol + stab", data = lab2).fit()
print(m4.summary())

Покажем на графике, как изменяется предельный эффект демократии и его значимость в зависимости от "условия" (freedom of press). Проинтерпретируйте график.

In [ ]:
cov_m4 = np.asmatrix(m4.cov_params())
fp_values = np.linspace(lab2.fp.min(), lab2.fp.max())
marg_effect_m4 = m4.params[1] + m4.params[3]*fp_values
se2 = (cov_m4[1,1] + fp_values**2*(cov_m4[3,3]) + 2*fp_values*cov_m4[1,3])**(.5)
fig, axes = mpl.subplots()
mpl.plot(fp_values, marg_effect_m4, 'b')
mpl.plot(fp_values, marg_effect_m4 - norm.ppf(0.975)*se2, 'b--')
mpl.plot(fp_values, marg_effect_m4 + norm.ppf(0.975)*se2, 'b--')
axes.set_xlabel('Freedom of press')
axes.set_ylabel('Marginal effect of democracy on corruption')
mpl.axhline(y=0, color ='r', linestyle = '--')
mpl.title('Dependence of the marginal effect on freedom of press')
mpl.show()

Резюмируйте результаты проведенного анализа.